In [2]:
%pip install langchain
%pip install openai
%pip install chromadb
%pip install unstructured
%pip install tiktoken
%pip install GitPython

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached numpy-1.24.3-cp311-cp311-macosx_11_0_arm64.whl (13.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
argilla 1.7.0 requires numpy<1.24.0, but you have numpy 1.24.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
  Using cached numpy-1.23.5-cp311-cp311-macosx_11_0_arm64.whl (13.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are

In [10]:
from langchain.document_loaders import GitLoader

loader = GitLoader(
    clone_url="https://github.com/seal-io/docs",
    repo_path="./docs",
    branch="main",
)
docs = loader.load()
print(len(docs))

147


In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = GitLoader(repo_path="./docs", file_filter=lambda file_path: file_path.endswith(".md"))

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = text_splitter.split_documents(loader.load())
print(len(split_docs))

29


In [13]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = OpenAIEmbeddings()

vectordb = Chroma.from_documents(split_docs, embeddings, persist_directory='chorma_data')
vectordb.persist()

In [18]:
vectordb = Chroma(persist_directory='chorma_data', embedding_function=embeddings)

docs = vectordb.similarity_search("如何加密关键数据？", k = 1)
print(docs[0].page_content)

# 常见问题

## Q1: 如何加密重要的帐密信息？

重要的帐密信息是指用户通过【运维中心/全局密钥】或【应用管理/密钥】面板所创建的值内容，通过【连接器】面板所填入的账户信息等。

为保护这类数据信息，用户可以在启动Seal实例时，通过环境变量来加密这类重要的帐密信息。

> 注意：该配置只在初次启动Seal实例时生效，二次配置会导致系统启动失败。

```shell
# step 1: Get 128/256 bits key with OpenSSL.

openssl rand -hex 16
# openssl rand -hex 32


# step 2: Start Seal with the follwoing command.

sudo docker run -d --privileged --restart=always \
  -p 80:80 -p 443:443 \
  -e SERVER_DATA_SOURCE_DATA_ENCRYPTION=aesgcm:${the output from step 1} \
  <seal-container-image>

```

目前仅支持使用 [AES](https://en.wikipedia.org/wiki/Advanced_Encryption_Standard) [GCM](https://en.wikipedia.org/wiki/Galois/Counter_Mode) 模式，更多对称加密的支持，欢迎沟通联系。

## Q2: 如何修改 Seal 日志配置？

Seal包含以下几个级别的日志：`debug`, `info`, `warning`, `error` 和 `fatal`。

配置参数 `log-debug` 和 `log-verbosity` 用于控制Seal日志打印。参数 `log-debug` 配置为`true`将打印出所有级别的日志；配置为`false`，则打印出`info`, `warning`, `error` 和 `fatal`级别的日志。配置参数 `log-verbosity` 用于控制 Seal 日志 verbosity 的最高级别，将打印出所有低于该级别的日志。


In [17]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.2)

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectordb.as_retriever())
qa.run("如何加密关键数据？")

'您可以在启动 Seal 实例时，通过环境变量来加密重要的帐密信息。具体步骤如下：\n\n1. 使用 OpenSSL 生成 128/256 位的密钥，命令如下：\n\n   ```\n   openssl rand -hex 16\n   # openssl rand -hex 32\n   ```\n\n2. 使用以下命令启动 Seal：\n\n   ```\n   sudo docker run -d --privileged --restart=always \\\n     -p 80:80 -p 443:443 \\\n     -e SERVER_DATA_SOURCE_DATA_ENCRYPTION=aesgcm:${the output from step 1} \\\n     <seal-container-image>\n   ```\n\n目前仅支持使用 AES GCM 模式进行加密，更多对称加密的支持，可以联系 Seal 官方进行沟通。请注意，该配置只在初次启动 Seal 实例时生效，二次配置会导致系统启动失败。'